In [1]:
import pandas as pd
import requests
import json
import os
import time
from concurrent.futures import ThreadPoolExecutor

In [2]:
def read_json(json_file_path, chunk_size=1000, max_rows=1000000):
    # Create an empty DataFrame to store the results
    result_df = pd.DataFrame()
    
    # Iterate over chunks
    for chunk_df in pd.read_json(json_file_path, lines=True, chunksize=chunk_size, encoding='utf-8'):
        # Filter by city names (replace 'your_city' with the actual city name)
        filtered_chunk = chunk_df[chunk_df['city'] == 'Philadelphia']
    
        # Append the filtered chunk to the result DataFrame
        result_df = pd.concat([result_df, filtered_chunk], ignore_index=True)
    
        # Break the loop if the total number of rows exceeds max_rows
        if len(result_df) >= max_rows:
            break
    return result_df

In [3]:
# df_reviews = pd.read_json("yelp_dataset/yelp_academic_dataset_review.json", lines=True, nrows=100)
# df_business = pd.read_json("yelp_dataset/yelp_academic_dataset_business.json", lines=True, nrows=10000)

Im trying to connect it to the json, but im on my local and working with the remote repo directly, so it cant work this way, also the Jupyter Notebook cannot use __file__


In [4]:
# If running in a Jupyter notebook cell, use the notebook's path
try:
    script_dir = os.path.dirname(os.path.abspath(__file__))
except NameError:
    # If __file__ is not defined (running as a Jupyter notebook cell)
    # use the current notebook's directory
    script_dir = os.path.abspath(".")

# Join the script directory with the relative path to the file
file_path = os.path.join(script_dir, "yelp_dataset/yelp_academic_dataset_business.json")

# Print the current working directory and the file path
print("Current working directory:", os.getcwd())
print("File path:", file_path)

# Check if the file exists
print("File exists:", os.path.exists(file_path))

Current working directory: C:\Users\PC\AppData\Local\Programs\Microsoft VS Code
File path: C:\Users\PC\AppData\Local\Programs\Microsoft VS Code\yelp_dataset/yelp_academic_dataset_business.json
File exists: False


Loading Businesses in Phili that are restaurants

In [5]:
# Set your Yelp Fusion API key here
API_KEY = '7THbPOimsD2p1860GmCA0YPxzRP6D1LgcbzmV9sY1srKXGuspoL0lKzAVAroIDL9g_AKrtBXz_3wQ8kA1r5iAipaFgwgIwSwekcB_2_W-cMBCo20bkd5jKxuYp9XZXYx'

# Yelp Fusion API endpoint for business search
BUSINESS_ENDPOINT = 'https://api.yelp.com/v3/businesses/search'

# Parameters for the API request (you can customize these)
search_parameters = {
    'term': 'restaurant',
    'location': 'Philadelphia',
    'limit': 50  # You can adjust this based on your needs
}

# Set up the headers with the API key for business search
headers = {
    'Authorization': f'Bearer {API_KEY}'
}

# Make the API request for business search
search_response = requests.get(BUSINESS_ENDPOINT, params=search_parameters, headers=headers)

# Check if the request for business search was successful (status code 200)
if search_response.status_code == 200:
    # Parse the JSON response for business search
    businesses_data = search_response.json()

    # Create a DataFrame directly from the 'businesses' data
    df_businesses = pd.DataFrame(businesses_data['businesses'])

    # Print column names
    print("Column Names:")
    print(df_businesses.columns)

    # Print the number of rows
    print(f"Number of Rows: {len(df_businesses)}")

else:
    print(f"Error: {search_response.status_code}, {search_response.text}")

Column Names:
Index(['id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count',
       'categories', 'rating', 'coordinates', 'transactions', 'price',
       'location', 'phone', 'display_phone', 'distance'],
      dtype='object')
Number of Rows: 50


In [6]:
df_businesses.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,6ajnOk0GcY9xbb5Ocaw8Gw,barbuzzo-philadelphia,Barbuzzo,https://s3-media2.fl.yelpcdn.com/bphoto/DORymf...,False,https://www.yelp.com/biz/barbuzzo-philadelphia...,3216,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.5,"{'latitude': 39.94999, 'longitude': -75.16216}",[delivery],$$,"{'address1': '110 S 13th St', 'address2': '', ...",+12155469300,(215) 546-9300,6942.869179
1,iksVwRfpWymIUUFqw0tXpw,chubby-cattle-philadelphia-5,Chubby Cattle,https://s3-media3.fl.yelpcdn.com/bphoto/gFMPEE...,False,https://www.yelp.com/biz/chubby-cattle-philade...,1485,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",4.5,"{'latitude': 39.95498, 'longitude': -75.15622}","[pickup, delivery, restaurant_reservation]",$$,"{'address1': '146 N 10th St', 'address2': '', ...",+18666228853,(866) 622-8853,6197.631405
2,i_FWONQD1ZBqrNE2b-M5Ug,talulas-garden-philadelphia,Talula's Garden,https://s3-media3.fl.yelpcdn.com/bphoto/dMuRtB...,False,https://www.yelp.com/biz/talulas-garden-philad...,2223,"[{'alias': 'newamerican', 'title': 'New Americ...",4.5,"{'latitude': 39.9473272217862, 'longitude': -7...",[delivery],$$$,"{'address1': '210 W Washington Sq', 'address2'...",+12155927787,(215) 592-7787,6835.442846
3,vUrTGX_7HxqeoQ_6QCVz6g,suraya-philadelphia-2,Suraya,https://s3-media1.fl.yelpcdn.com/bphoto/H6vumh...,False,https://www.yelp.com/biz/suraya-philadelphia-2...,1443,"[{'alias': 'lebanese', 'title': 'Lebanese'}, {...",4.5,"{'latitude': 39.9736865005167, 'longitude': -7...",[delivery],$$$,"{'address1': '1528 Frankford Ave', 'address2':...",+12153021900,(215) 302-1900,3471.867459
4,v45E6gg6MrAziTepkiWByA,harpers-garden-philadelphia,Harper's Garden,https://s3-media4.fl.yelpcdn.com/bphoto/NqyB3E...,False,https://www.yelp.com/biz/harpers-garden-philad...,727,"[{'alias': 'newamerican', 'title': 'New Americ...",4.0,"{'latitude': 39.9523171, 'longitude': -75.1700...",[delivery],$$,"{'address1': '31 S 18th St', 'address2': '', '...",+12678868552,(267) 886-8552,7112.873384


In [7]:
# Extract business IDs from df_businesses
business_ids = df_businesses['id'].tolist()

ok business id is a concrete ENDPOINT for the API call, so i have to call 50 times for 50 businesses, there is no mapping involved. Lets try again

That is alot for the API to handle, so we are using a delay between requests, also 500 calls for 1 endpoint per hour and 25 000 per day.

In [11]:
# Yelp Fusion API endpoint for business reviews
REVIEWS_ENDPOINT = 'https://api.yelp.com/v3/businesses/{business_id}/reviews'

# Maximum number of reviews to fetch per business
max_reviews_per_business = 100  # Adjust this value based on the API limit

# List to store review data
all_reviews = []

# Function to fetch reviews for a business ID
def get_reviews(business_id):
    reviews_response = requests.get(REVIEWS_ENDPOINT.format(business_id=business_id), headers=headers, params={'limit': max_reviews_per_business})
    
    if reviews_response.status_code == 200:
        reviews_data = reviews_response.json()
        
        # Extract and append reviews to the list
        business_reviews = reviews_data.get('reviews', [])
        return business_reviews
    else:
        print(f"Error loading reviews for business {business_id}: {reviews_response.status_code}, {reviews_response.text}")
        return []

# Introduce a delay between requests
delay_between_requests = 10000  # You can adjust this value based on your needs

# Use ThreadPoolExecutor to parallelize the API requests
with ThreadPoolExecutor(max_workers=5) as executor:  # You can adjust max_workers based on your needs
    # Map the function over the list of business IDs
    all_reviews = list(executor.map(get_reviews, business_ids))
    # Introduce a delay between requests
    time.sleep(delay_between_requests)

# Flatten the list of lists into a single list of reviews
all_reviews = [review for sublist in all_reviews for review in sublist]

# Create a DataFrame from the list of reviews
df_reviews = pd.DataFrame(all_reviews)


Error loading reviews for business ctHjyadbDQAtUFfkcAFEHw: 429, {"error": {"code": "TOO_MANY_REQUESTS_PER_SECOND", "description": "You have exceeded the queries-per-second limit for this endpoint. Try reducing the rate at which you make queries."}}
Error loading reviews for business Ipkx4Sa7ybn8C6LtTqTztw: 429, {"error": {"code": "TOO_MANY_REQUESTS_PER_SECOND", "description": "You have exceeded the queries-per-second limit for this endpoint. Try reducing the rate at which you make queries."}}
Error loading reviews for business 4NYZ-PHH7GDgjfIO7_Vfgg: 429, {"error": {"code": "TOO_MANY_REQUESTS_PER_SECOND", "description": "You have exceeded the queries-per-second limit for this endpoint. Try reducing the rate at which you make queries."}}
Error loading reviews for business HTgKfmmlzgE43trhntv8-w: 429, {"error": {"code": "TOO_MANY_REQUESTS_PER_SECOND", "description": "You have exceeded the queries-per-second limit for this endpoint. Try reducing the rate at which you make queries."}}
Erro

In [10]:
# Print column names and number of rows for the reviews DataFrame
print("\nReviews DataFrame:")
print("Column Names:")
print(df_reviews.columns)
print(f"Number of Rows: {len(df_reviews)}")

# Display the first few rows of the reviews DataFrame
print("\nFirst Few Rows of Reviews DataFrame:")
print(df_reviews.head())


Reviews DataFrame:
Column Names:
RangeIndex(start=0, stop=0, step=1)
Number of Rows: 0

First Few Rows of Reviews DataFrame:
Empty DataFrame
Columns: []
Index: []


SOMETHING

In [7]:
df_business.city.value_counts()

NameError: name 'df_business' is not defined

In [29]:
df_business.to_json('yelp_dataset/yelp_filtered_philadelphia_business.json', orient='records')